In [1]:
from collections import defaultdict
import os
import json

def read_dir(data_dir):
    clients = []
    groups = []
    data = defaultdict(lambda: None)

    files = os.listdir(data_dir)
    files = [f for f in files if f.endswith('.json')]
    for f in files:
        file_path = os.path.join(data_dir, f)
        with open(file_path, 'r') as inf:
            cdata = json.load(inf)
        clients.extend(cdata['users'])
        if 'hierarchies' in cdata:
            groups.extend(cdata['hierarchies'])
        data.update(cdata['user_data'])

    clients = list(sorted(data.keys()))
    return clients, groups, data


def read_data(train_data_dir, test_data_dir):
    train_clients, train_groups, train_data = read_dir(train_data_dir)
    test_clients, test_groups, test_data = read_dir(test_data_dir)

    assert train_clients == test_clients
    assert train_groups == test_groups

    return train_clients, train_groups, train_data, test_data

_, _, train_data, test_data = read_data("data/femnist/train", "data/femnist/test")

In [2]:
for i in range(9):
    train_file = open(f"./data/femnist/train/node{i}.json", "w")
    train_file.write(json.dumps(train_data[str(i)]))
    test_file = open(f"./data/femnist/test/node{i}.json", "w")
    test_file.write(json.dumps(test_data[str(i)]))